In [2]:
# os ifadesi, Python’da işletim sistemiyle etkileşim kurmanı sağlayan yerleşik (built-in) bir kütüphanedir.
# Bir dosya oluşturduğunda, sen bu dosyanın içeriğini fark etmez bunu bir hash algoritması ile işlersin, dosyanın içeriğinden -
# türeyen benzersiz bir dijital imza (hash değeri) elde edersin.Dosyanın içine tek bir harf bile eklersen, hash değeri tamamen farklı olur.-
# hashlib kütüphanesi, birçok hash algoritmasını destekler.SHA-256, BLAKE2s,SHA3-256, SHA3-512
# o zaman dict list gibi veriyapıları sadece python algılayabiliyor o yüzden de verileri paylaşmak için evrensel bir formatta olan-
#json kullanmalıyız bu sayede paylaşılması saklanması ve anlaşılması daha kolay olur.
# argparse, Python programını terminalden çalıştırırken dışarıdan veri (argüman) vermeni sağlar.
# difflib İki metin, liste veya dosya arasındaki benzerlikleri ve farkları bulmak için kullanılır.
# pathlib, Python'da dosya ve klasör yollarıyla çalışmayı kolaylaştıran bir modüldür.
# watchdog Dosya ve klasörlerdeki değişiklikleri gerçek zamanlı izlemek için kullanılır.Observer Klasörü izlemeye yarayan gözlemci (watcher) nesnesi
# FileSystemEventHandler Dosyada bir olay olduğunda ne yapılacağını tanımlayan temel sınıf
# smtplib, ssl, EmailMessage → mail gönderimi için

import os
import hashlib
import json
import argparse
import time
import difflib
import smtplib
import ssl
from email.message import EmailMessage
from pathlib import Path
from datetime import datetime
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


SMTP_HOST = "smtp.gmail.com" # gmail smtp ayarları
SMTP_PORT = 587 # tls için
SMTP_USER = ""   # senin Gmail adresin
SMTP_PASS = ""         # Google’dan aldığın App Password
EMAIL_FROM = ""  # genelde SMTP_USER ile aynı
EMAIL_TO   = [""]  # mailin gideceği adres(ler)
EMAIL_ENABLE = True 

# spam ve uzun mailleri engellemek için 
MAX_DIFF_LINES_IN_MAIL = 200

# fim_baseline.json: Dosyaların ilk durumlarını tutar.
# fim_log.txt: Tespit edilen değişiklikleri loglar.

BASELINE_FILE = "fim_baselines_new_1.json"
LOG_FILE = "fim_logs_new_1.txt"

# mail gönderme programı, smtp ye bağlan ,tls ile güvenli oturum aç, app password ile giriş yap 
# bu fonksiyon mail başlığı , mail içeriği string alır
def send_email(subject: str, body: str):
    if not EMAIL_ENABLE:
        return
    """ kimden kime ve konu kısmını doldurur"""
    try:
        msg = EmailMessage()
        msg["From"] = EMAIL_FROM
        msg["To"] = ", ".join(EMAIL_TO)
        msg["Subject"] = subject
        """verilen yazı mailin gövdesine eklenir"""
        msg.set_content(body)
        """Güvenli TLS bağlantısı için bir SSL context nesnesi hazırlanıyor."""
        context = ssl.create_default_context()
        with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
            server.starttls(context=context)
            server.login(SMTP_USER, SMTP_PASS)
            server.send_message(msg)
    except Exception as e:
        # Mail hatası olsa bile FIM durmasın
        log_event(f"EMAIL_ERROR: {e}")

def notify_event(event_type: str, path: str, details: str | None = None):
    """Tekil olaylar için bildirim maili."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    subject = f"[FIM] {event_type} @ {os.path.basename(path)}"
    body = [
        f"Time     : {timestamp}",
        f"Event    : {event_type}",
        f"Path     : {path}",
    ]
    if details:
        body.append("")
        body.append("Details:")
        body.append(details)

    send_email(subject, "\n".join(body))


# datetime.now() Şu anki tarihi ve saati alır. ve .strftime uygun zaman biçimine çevirir.
# Append  Dosya varsa içindekileri silmez, en sona ekleme yapar.
def log_event(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a") as f:
        f.write(f"[{timestamp}] {message}\n")
    print(f"[{timestamp}] {message}")


def calculate_file_info(path):
    try:
        stats = os.stat(path)
        return {
            "hash": calculate_hash(path),
            "size": stats.st_size,
            "modified": stats.st_mtime,
            "created": stats.st_ctime,
            
        }
    except (FileNotFoundError, PermissionError):
        return None

# hash değeri için "rb" şart çünkü biz dosyayı olduğu gibi (byte byte) okumalıyız.
# okunan veriyi hem chunk’a atar hem de “döngü devam etsin mi?” kontrolünde kullanır. 
# Boş byte dizisi (b'') geldiğinde dosyanın sonuna gelmiş oluruz ve döngü biter.
# sha256.update(chunk) her parçayı hash’e dahil eder. Böylece çok büyük dosyaları da düşük bellekle işleyebiliriz
# hexdigest 256 bit, hexadecimal (16’lık sayı sistemi) ile yazılmış bir string olarak gösteriliyor.
#Bu yüzden 64 karakter uzunluğunda (0–9 ve a–f harfleri içerir).
def calculate_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, 'rb') as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

# os.walk alt klasörlere otomatik iner.ve onları da tek tek root olarak döndürür.
# root şu an gezilen klasörün yolu
# files o klasördeki dosya isimlerinin listesi
#os.path.join(root, file) → dosyanın tam yolunu birleştirir.
# Her dosya için calculate_file_info fonksiyonu çağrılıyor. Bu fonksiyon dosyanın hash, boyut, tarih bilgilerini döndürür.
# Bu fonksiyonun amacı verilen klasördeki tüm dosyaların bir “fotoğrafını” çıkarmaktır.
def generate_baseline(directory):
    baseline = {}
    for root, _, files in os.walk(directory):
        for file in files:
            full_path = os.path.abspath(os.path.join(root, file))
            file_info = calculate_file_info(full_path)
            if file_info:
                baseline[full_path] = file_info
    return baseline

# baseline sözlüğünü alır. BASELINE_FILE dosyasını açar. İçeriği JSON formatında dosyaya yazar.
def save_baseline(baseline):
    with open(BASELINE_FILE, "w") as f:
        json.dump(baseline, f, indent=4)

# Eğer BASELINE_FILE yoksa {} (boş dict) döner. Varsa dosyayı açar, JSON’dan okur, dict olarak geri döner.
def load_baseline():
    if not os.path.exists(BASELINE_FILE):
        return {}
    with open(BASELINE_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


#parametre olarak eski ve yeni baseline alır
# eskı base ve yeni base doyları yolu için küme işlemi yapabilmek için sete çevirir.
# yeni olup eskide olmayanları bulmak için veri yapısının set olması gerekiyor Fark (-), kesişim (&), birleşim (|)
# eski ve yeni hash birbirine eşit değilse dosya içeriği değişmiş demektir.
# Fonksiyonun amacı Yeni dosya var mı,Silinmiş dosya var mı,İçeriği değişmiş dosya var mı
def compare_states(old, new):
    added = []
    modified = []
    deleted = []

    old_keys = set(old.keys())
    new_keys = set(new.keys())

    for path in new_keys - old_keys:
        added.append(path)

    for path in old_keys - new_keys:
        deleted.append(path)

    for path in new_keys & old_keys:
        if old[path]["hash"] != new[path]["hash"]:
            modified.append(path)

    return added, modified, deleted

#difflib.unified_diff → iki metin arasındaki farkları “git diff” tarzında üretir.
# dosya farklı sistemlerde çalıştığızaman sorun yaşamaması için UTF-8 neredeyse tüm dillerin karakterlerini dekleyen kulllanıldı
#Yani bu fonksiyonda amacım dosya içinde hangi satırlar değişti
def show_file_diff(old_path, new_path):
    try:
        with open(old_path, "r", encoding="utf-8") as f1:
            old_lines = f1.readlines()
    except FileNotFoundError:
        old_lines = []

    try:
        with open(new_path, "r", encoding="utf-8") as f2:
            new_lines = f2.readlines()
    except FileNotFoundError:
        new_lines = []

    diff = difflib.unified_diff(old_lines, new_lines, fromfile="OLD", tofile="NEW", lineterm="")
    diff_output = "\n".join(diff)

    if diff_output.strip():
        print("\n--- Dosya Değişiklikleri (Satır Bazında) ---")
        print(diff_output)
        log_event("DIFF:\n" + diff_output)

        # Mail için kısaltılmış diff
        short_diff = "\n".join(diff_lines[:MAX_DIFF_LINES_IN_MAIL])
        return short_diff
        
class FIMEventHandler(FileSystemEventHandler):
    def __init__(self, baseline):
        self.baseline = baseline

    def on_modified(self, event):
        if not event.is_directory:
            path = os.path.abspath(event.src_path)
            new_info = calculate_file_info(path)
            old_info = self.baseline.get(path)

            if old_info and new_info:
                if old_info["hash"] != new_info["hash"]:
                    log_event(f"MODIFIED: {path}")
                    show_file_diff(path, path)
                    self.baseline[path] = new_info
                    save_baseline(self.baseline)
                    # E-posta bildirimi
                    details = None
                    if diff_text:
                        details = f"Diff (first {MAX_DIFF_LINES_IN_MAIL} lines):\n{diff_text}"
                    else:
                        details = "Content changed (hash differs), diff not available."
                    notify_event("MODIFIED", path, details)

    def on_created(self, event):
        if not event.is_directory:
            path = os.path.abspath(event.src_path)
            new_info = calculate_file_info(path)
            if new_info:
                log_event(f"CREATED: {path}")
                self.baseline[path] = new_info
                save_baseline(self.baseline)
                # E-posta bildirimi
                details = f"size={new_info['size']} bytes"
                notify_event("CREATED", path, details)

    def on_deleted(self, event):
        if not event.is_directory:
            path = os.path.abspath(event.src_path)
            if path in self.baseline:
                log_event(f"DELETED: {path}")
                self.baseline.pop(path)
                save_baseline(self.baseline)
                # E-posta bildirimi
                notify_event("DELETED", path, "File removed from filesystem and baseline.")


def monitor_directory(directory):
    baseline = load_baseline()
    event_handler = FIMEventHandler(baseline)
    observer = Observer()
    observer.schedule(event_handler, directory, recursive=True)
    observer.start()
    log_event(f"Real-time monitoring started on: {directory}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        log_event("Monitoring stopped.")
    observer.join()


def scan_directory(directory):
    old = load_baseline()
    new = generate_baseline(directory)
    added, modified, deleted = compare_states(old, new)

    print("\nScan Results:")
    for path in added:
        print(f"[+] ADDED: {path}")
        log_event(f"ADDED: {path}")
    for path in modified:
        print(f"[~] MODIFIED: {path}")
        log_event(f"MODIFIED: {path}")
    for path in deleted:
        print(f"[-] DELETED: {path}")
        log_event(f"DELETED: {path}")
    # Özet mail (değişiklik varsa)
    if (added or modified or deleted) and EMAIL_ENABLE:
        lines = ["Scan summary:", f"Directory: {directory}", ""]
        if added:
            lines.append("ADDED:")
            lines.extend([f"  - {p}" for p in added])
        if modified:
            lines.append("")
            lines.append("MODIFIED:")
            lines.extend([f"  - {p}" for p in modified])
        if deleted:
            lines.append("")
            lines.append("DELETED:")
            lines.extend([f"  - {p}" for p in deleted])

        send_email("[FIM] Scan Summary", "\n".join(lines))

    save_baseline(new)


def main():
    # TEST: Jupyter'da çalıştırmak için örnek değer
    class Args:
        mode = "scan"   # "init" | "scan" | "monitor"
        directory = r"C:\Users\MONSTER\Documents\tei_try\fim"  # test edeceğin klasör

    args = Args()
    directory = os.path.abspath(args.directory)

    if args.mode == "init":
        baseline = generate_baseline(directory)
        save_baseline(baseline)
        log_event(f"Baseline created for {directory}")
    elif args.mode == "scan":
        scan_directory(directory)
    elif args.mode == "monitor":
        monitor_directory(directory)



if __name__ == "__main__":
    main()



Scan Results:
[+] ADDED: C:\Users\MONSTER\Documents\tei_try\fim\27.08.2025.txt
[2025-08-27 13:22:05] ADDED: C:\Users\MONSTER\Documents\tei_try\fim\27.08.2025.txt
[-] DELETED: C:\Users\MONSTER\Documents\tei_try\fim\hello.txt
[2025-08-27 13:22:05] DELETED: C:\Users\MONSTER\Documents\tei_try\fim\hello.txt
